## Code for Noah's Hypermutation Project<br><br/>
Run the code in this notebook to generate the files used for analysis and the figures themselves.  <br><br/>
Code runs by referencing paths in the file: fileDirectory.txt
The first section of this notebook describes how to create files for analysis.  The second part creates the figures.  The final files are already included so you can skip ahead if you would like

Environment:<br>
Python 2.7.15 |Anaconda custom (64-bit)| (default, Dec 14 2018, 13:10:39)<br>
R version 3.5.1 (2018-07-02) -- "Feather Spray"<br>


In [2]:
import sys
import os

#TODO make 
sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
filePathDict = configuration_util.get_all_files_path_dict()

<hr style="border:2px solid gray"> </hr>

## Files not created by me
### Files
* **CANCER_TYPE_INFO** information about TMB, cancer type and msi status, generated by Chai
* **TCGA_CANCER_TYPE_INFO** information about TCGA cancer types, from cluster
* **GENE_LENGTH_INFO** information about coding sequence lengths for genes, from cluster, originall ensemble?
* **DEP_MAP_DATA** depmap scores for all genes.  Taken from https://ndownloader.figshare.com/files/22629068 -->achilles_gene_effect.csv.  CRISPR data
* **PHASING_DATA** generated by Alex G, using his phasing pipeline on IMPACT data
* **HOTSPOT_DATA** summary of all known hotspots from cancerhotspots.org
* **MICROSATELLITE_INFORMATION** summary of all microsatellite sites in IMPACT panel.  Generated by Craig
* **NUCLEOSOME_DYAD_POSITIONS** summary of nucleosome dyad positions, taken from Alex G, before that from Nuria's paper

### Mafs
* **IMPACT_BASE_MAF** impact filtered mutations from portal, annotated from Novemeber 2019
* **IMPACT_BASE_MAF_WITH_SYNONYMOUS** impact unfiltered mutations from portal
* **ALL_EXOME_MAF** maf combining all Exome recapture mutations and all TCGA mutations
* ****


<hr style="border:2px solid gray"> </hr>

# Generation of analysis files

## Files generated on the cluster

#### Signatures
run <br>
produces **IMPACT_SIGNATURE_DECOMPOSITIONS**
run <br>
produces **TCGA_SIGNATURE_DECOMPOSITIONS**
run <br>
produces **EXOME_RECAPTURE_SIGNATURE_DECOMPOSITIONS**
run <br>
produces **CLONAL_VS_SUBCLONAL_MUTATIONAL_SIGNATURES**

**Calculation of expected rate of hotspot, oncogenic and truncating mutation**

In [ ]:
#On the cluster run:
#command generates a vcf with all positions
cmdAllVcfPos = '/opt/common/CentOS_7-dev/bin/bcftools mpileup --skip-indels --fasta-ref /ifs/depot/pi/resources/genomes/GRCh37/fasta/b37.fasta --regions-file /ifs/depot/pi/resources/roslin_resources/targets/IMPACT468/b37/IMPACT468_b37_targets.bed --output impact468_snps.vcf /ifs/res/pi/Proj_09221.6f74bdee-146b-11e9-a68d-645106ef9e4c/bam/s_C_P3U7DR_N001_d.Group3.rg.md.abra.printreads.bam'
#add in every possible alt allele
cmdAddAltAlleles = 'python add_all_variants_to_vcf.py ~/impact468_snps.vcf ~/friedman/myAdjustedDataFiles/impact468_all_possible_snps.vcf'
#use commands of the form below to cut vcf up into chromosome sized chunks
'bcftools view /home/friedman/friedman/myAdjustedDataFiles/impact468_all_possible_snps_sorted.vcf.gz --regions 4 -o /juno/work/taylorlab/friedman/myAdjustedDataFiles/allPossibleMutationsInIMPACT/vcfs/chr4AllMuts.vcf'
#convert it to a maf: (note writes a .sh file)
#run vcf2Maf
python run_vcf2Maf_on_all_possibleSnpsMaf.py
#annotateMafs
python run_maf2Maf_on_all_possibleSnpsMaf.py

## Pentanucleotide annotated mafs

**Classification of hypermutated tumors**<br><br/>


In [ ]:
cmd = ' '.join(['python', filePathDict['SCRIPT_DEFINE_HYPERMUTATION_THRESHOLDS'])
os.popen(cmd)

**Calculation of expected rate of hotspot, oncogenic and truncating mutation**

In [ ]:
#calculate signature propensity to cause drivers
cmdGet

**Estimating the expected number of consequential mutations: Nucleotide context method**<br/>
Enumerate all possible mutation mafs

In [ ]:
cmd = ''

**Identifying tumor subclones and copy number**<br><br/>
On our compute cluster we ran the FACETs algorithm with the following parameters:?
On the cluster we then summarize these results through a shared codebase: python myUtils/create_cncf_or_rdata_file_list.py then python prepare_maf_anno_commands.py then /juno/work/taylorlab/friedman/myUtils/runMafAnno.sh which calls myUtils/runAnnotateMaf.R then combine results with python concat_maf_util.py /juno/work/taylorlab/friedman/myAdjustedDataFiles/mafAnnoRuns/annotatedMafs /juno/work/taylorlab/friedman/myAdjustedDataFiles/mafAnnoRuns/combined_cncf_annotated.maf

<hr style="border:2px solid gray"> </hr>

## Files generated locally

In [ ]:
#adjust clonality calls for hypermutated tumors with flat genomes:
#this command takes ~1hr to run
cmd = ' '.join(['python', filePathDict['SCRIPT_ADJUST_CLONALITY_CALLS'],
                filePathDict['files/mafs/combined_cncf_annotated.maf'], filePathDict['IMPACT_MAF_WITH_ADJUSTED_CLONALITY_ANNOTATION'], 'IMPACT'])
os.popen(cmd)

**DNDS ANALYSIS**

In [ ]:
#todo make sure the input and output commands are correct here
#make sure this script properly adjusts the file to run dnds
cmd = ' '.join(['RScript', filePathDict['SCRIPT_RUN_DNDS_CV'], filePathDict['IMPACT_BASE_MAF_WITH_SYNONYMOUS']])

#results are written to:
''
#TODO THE DNDS ANALYSIS script now has an annoying software bug


**Mutation Atribution**<br><br/>
Attributes mutations to signatures as done in Alex's paper.  We use different thresholds for attribution to account for the different characteristics of hypermutated cases

In [4]:
cmd = ' '.join(['python', filePathDict['SCRIPT_ATTRIBUTE_MUTATIONS_TO_SIGNATURES'],
                filePathDict['SIGNATURE_SPECTRUM'], '.', '1', '10',
                filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'attributedMutations.tsv',
                'agingIsAlwaysPresent', 'doSmokingCorrection'])
#os.popen(cmd)

#ALERT THIS IS BUGGY
print cmd

python /Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/attribute_mutations_to_signatures.py /Users/friedman/Desktop/hypermutationProjectFinal/files/infoFiles/Stratton_signatures30.txt . 1 10 /Users/friedman/Desktop/hypermutationProjectFinal/files/infoFiles/impactSignatureCalls_Nov20_2019.tsv attributedMutations.tsv agingIsAlwaysPresent doSmokingCorrection


**PHASING**<br><br/>
results are generated using methods described in: ALEX's paper

**Constructing Phylogenetic Trees**

# Generate Figures

**FIGURE 1**

In [ ]:
#FIGURE 1A 
#FIGURE 1B
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_1bc']
#FIGURE 1C
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_1bc']
#FIGURE 1D
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_1d']
#FIGURE 1E
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_1e']
#FIGURE 1F
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_1f']

**FIGURE 2**

In [ ]:
#FIGURE 2A 
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_2ac']
#FIGURE 2B
#FIGURE 2C
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_2ac']
#FIGURE 2D
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_2d']
#FIGURE 2E

**FIGURE 3**

In [ ]:
#FIGURE 3A
#FIGURE 3B
#FIGURE 3C
#FIGURE 3D
#FIGURE 3E
filePathDict['SCRIPT_GENERATE_PLOT_FIGURE_3e']

**FIGURE 4**